In [ ]:
import pandas as pd
import pyodbc
import numpy as np
import plotly.express as px
from datetime import datetime
import statsmodels.api as sm
# import statsmodels.tsa.api as tsa

pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = 50
pd.options.display.max_rows = 30

def file_to_string(fileName):
    file = open(fileName,'r')
    string = file.read()
    file.close()
    return string

def run_query(query):
    cnxn = pyodbc.connect('DSN=edp-workbench-cshub', autocommit=True)
    df = pd.read_sql_query(query,cnxn)
    cnxn.close()
    return df
    
def timestamp():
    return datetime.now().strftime('%Y-%m-%d %I:%M:%S %p')

def add_conditionals(df):
    df['Date']=pd.to_datetime(df['Date'])
    df['TestGroup'] = (df['Employee'].isin((
        '364717','426097','547655','552121','554487','575731'
        '569375','572247','572815','572909','573190','580895'
        '573192','573276','573573','573585','575996','580911'
        '576565','576911','579162','581139','581275','581015'))).astype(int)
    df['AfterExpertAssistLaunch']=(df['Date']>=datetime(2023,9,25)).astype(int)
    df['ExpertAssistOnForThisUser'] = df['AfterExpertAssistLaunch']*df['TestGroup']
    df['Day'] = df['Date'].dt.weekday
    days = {0:'Monday',1:'Tuesday',2:'Wednesday',3:'Thursday',4:'Friday',5:'Saturday',6:'Sunday'}
    for day in days:
        df[days[day]]=(df['Day']==day).astype(int)
    df['Week']=df['Date'].dt.isocalendar().week
    for week in df['Week'].unique():
        df['Week '+str(week)]= (df['Week']==week).astype(int)
    for queue in df['TaskQueue'].unique():
        df[queue] = (df['TaskQueue']==queue).astype(int)
    df['HandleTimeZ']=(df['HandleTime']-df['HandleTime'].mean())/df['HandleTime'].std()
    df['Log(HandleTime)']=np.log(df['HandleTime'].replace(0,1))
    df['Log(HandleTime)Z']=(df['Log(HandleTime)']-df['Log(HandleTime)'].mean())/df['Log(HandleTime)'].std()
    df['Constant'] =1
    return df

PILOT_EXPERTS = """
    '293932','321930','364717','393198','426097',
    '536491','538855','547381','547655','548026',
    '548646','549667','550446','552121','552402',
    '554243','554487','561761','567519','567620',
    '568127','568568','568651','569153','569375',
    '569498','570227','572247','572815','572909',
    '573190','573192','573276','573573','573585',
    '574166','575731','575996','576565','576601',
    '576911','577073','577246','578299','579162',
    '579630','580895','580911','581015','581139',
    '581145','581275','581396','581694' 
    """


In [2]:
dfp = run_query(file_to_string('AHT.SQL').replace('$pilot_experts$', PILOT_EXPERTS ))
# dfp = dfp[dfp['HandleTime']!=0]

# Diference in Diferences Method

In [3]:
df = dfp.copy()
df.head()
df = add_conditionals(df)
df['Group'] = np.where(df['TestGroup'],'Test','Control')
df['Period']=np.where(df['AfterExpertAssistLaunch'],'PostLaunch','PreLaunch')

pivot = pd.pivot_table(df,columns=['AfterExpertAssistLaunch','TestGroup'],index='TaskQueue',values='HandleTime',margins=True)
display(pivot.sort_index())
del pivot

pivot = pd.pivot_table(df,columns=['Period','Group'],index='TaskQueue',values='HandleTime',margins=True)
for col in pivot['PostLaunch'].columns:
        pivot['% Difference',col]=((pivot['PostLaunch',col]/pivot['PreLaunch',col])-1)
        pivot['Abs Difference',col]=((pivot['PostLaunch',col]-pivot['PreLaunch',col]))
        pivot['Total Calls in Queue',col]=df[df['Group']==col]['TaskQueue'].value_counts()
        pivot.loc['All']['Total Calls in Queue',col]=pivot['Total Calls in Queue',col].sum()
pivot['% Difference','B/-(W) Than Control']=pivot['% Difference','Control']-pivot['% Difference','Test']
pivot['Abs Difference','B/-(W) Than Control']=pivot['Abs Difference','Control']-pivot['Abs Difference','Test']

pivot = pivot.sort_values(by=[('Total Calls in Queue','Test')],axis=0,ascending=False)
pivot = pivot.sort_index(axis=1,ascending=False)
del pivot['All']

display("AHT by Test Period and Group",pivot)
# pivot.to_excel('AHT.xlsx')
del pivot, df, col

AfterExpertAssistLaunch        0                 1               All
TestGroup                      0        1        0        1         
TaskQueue                                                           
All                     1,049.68 1,156.10 1,053.02 1,144.96 1,085.34
VZN_MTS_Bundle_EN_2233  1,025.14 1,142.56 1,033.47 1,130.53 1,066.16
VZN_Onboarding_2628     1,293.72 1,301.07 1,247.98 1,296.18 1,280.40

'AHT by Test Period and Group'

Period                 Total Calls in Queue          PreLaunch           \
Group                                  Test  Control      Test  Control   
TaskQueue                                                                 
All                                3,758.00 7,232.00  1,156.10 1,049.68   
VZN_MTS_Bundle_EN_2233             3,434.00 6,572.00  1,142.56 1,025.14   
VZN_Onboarding_2628                  324.00   660.00  1,301.07 1,293.72   

Period                 PostLaunch          Abs Difference          \
Group                        Test  Control           Test Control   
TaskQueue                                                           
All                      1,144.96 1,053.02         -11.14    3.34   
VZN_MTS_Bundle_EN_2233   1,130.53 1,033.47         -12.03    8.33   
VZN_Onboarding_2628      1,296.18 1,247.98          -4.89  -45.75   

Period                                     % Difference          \
Group                  B/-(W) Than Control         Test Control   
TaskQueue                                                         
All                                  14.48        -0.01    0.00   
VZN_MTS_Bundle_EN_2233               20.36        -0.01    0.01   
VZN_Onboarding_2628                 -40.86        -0.00   -0.04   

Period                                      
Group                  B/-(W) Than Control  
TaskQueue                                   
All                                   0.01  
VZN_MTS_Bundle_EN_2233                0.02  
VZN_Onboarding_2628                  -0.03

# Boxplots and numbers per group
df = dfp.copy()
df = add_conditionals(df)


px.box(x=df['TaskQueue'],y=df['HandleTime']).show()
pivot=pd.pivot_table(df,columns='TaskQueue',index='Date',values='HandleTime')
display(pivot.describe())
del pivot
display(pd.pivot_table(df,index='TaskQueue',values=['HandleTime']))
display('Percent of Each Task',df[df['TaskQueue'].unique()].mean())
del df

In [4]:
# Boxplots and numbers per week
df = dfp.copy()
df = add_conditionals(df)
df['Group'] = np.where(df['TestGroup'],'Test','Control')
px.box(df,x='Week',y='HandleTimeZ',color='TestGroup').show()
pivot=pd.pivot_table(df,columns='Week',index='Employee',values='HandleTimeZ',aggfunc=np.mean)
px.histogram(df,x='HandleTimeZ',nbins=20).show()
px.histogram(df,x='Log(HandleTime)Z',nbins=20,histnorm='probability').show()
display(pivot.describe())
del df, pivot

Week,37,38,39,40,41
count,49.00,48.00,43.00,41.00,30.00
mean,0.04,0.02,0.01,0.03,0.13
std,0.36,0.34,0.29,0.32,0.28
mad (mean),0.30,0.27,0.23,0.23,0.20
mad (median),0.28,0.24,0.15,0.18,0.14
min,-0.88,-0.71,-0.61,-0.66,-0.44
25%,-0.21,-0.27,-0.14,-0.18,-0.02
50%,-0.04,0.02,-0.02,0.01,0.09
75%,0.33,0.22,0.19,0.20,0.24
max,0.71,0.86,0.68,1.20,0.83


# Regression of Handle Time

In [5]:
df = dfp.copy()
df = add_conditionals(df)


# Look at only 2 queue types
df['Having ExpertAssist Effect on Onboarding']=df['VZN_Onboarding_2628']*df['ExpertAssistOnForThisUser']
df['Having ExpertAssist Effect on MTS Bundle']=df['VZN_MTS_Bundle_EN_2233']*df['ExpertAssistOnForThisUser']
df['Onboarding Post Launch']=df['VZN_Onboarding_2628']*df['AfterExpertAssistLaunch']
# df['MTS Bundle Post Launch']=df['VZN_MTS_Bundle_EN_2233']*df['AfterExpertAssistLaunch']
df['Week']=df['Week'].astype(int)
df['WeeksAgo']=-(df['Week']-df['Week'].max())
# display(df[['Date','Employee','TaskQueue',
#             'TestGroup','AfterExpertAssistLaunch','ExpertAssistOnForThisUser',
#             'HandleTime','Having ExpertAssist Effect on Onboarding',
#             'Having ExpertAssist Effect on MTS Bundle','WeeksAgo']].tail(25),df.columns)
model = sm.OLS(
    endog=df['HandleTime'],
    # endog=df['HandleTime'],
    exog=df[[
        'Having ExpertAssist Effect on Onboarding',
        'Having ExpertAssist Effect on MTS Bundle','Constant','TestGroup','WeeksAgo',
        "VZN_Onboarding_2628"
]]).fit()
display(model.predict(exog=[1,0,1,1,1,1])-model.predict(exog=[0,0,1,1,1,1]))
display(model.predict(exog=[0,1,1,1,1,0])-model.predict(exog=[0,0,1,1,1,0]))
display(model.summary(),model.summary2().tables[1])
del df, model

array([-66.36919989])

array([-6.94166816])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             HandleTime   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.006
Method:                 Least Squares   F-statistic:                     14.87
Date:                Tue, 10 Oct 2023   Prob (F-statistic):           1.43e-14
Time:                        13:06:25   Log-Likelihood:                -90911.
No. Observations:               10990   AIC:                         1.818e+05
Df Residuals:                   10984   BIC:                         1.819e+05
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Having ExpertAssist Effect on Onboarding   -66.3692     87.337     -0.760      0.447    -237.566     104.827
Having ExpertAssist Effect on MTS Bundle    -6.9417     37.025     -0.187      0.851     -79.517      65.633
Constant                                  1031.4772     24.951     41.340      0.000     982.568    1080.386
TestGroup                                  106.3853     25.673      4.144      0.000      56.061     156.709
WeeksAgo                                    -0.2836      8.914     -0.032      0.975     -17.757      17.189
VZN_Onboarding_2628                        225.0460     34.485      6.526      0.000     157.448     292.644
==============================================================================
Omnibus:                     5824.655   Durbin-Watson:                   2.002
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            54972.894
Skew:                           2.376   Prob(JB):                         0.00
Kurtosis:                      12.873   Cond. No.                         28.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

,Coef.,Std.Err.,t,P>|t|,[0.025,0.975]
Having ExpertAssist Effect on Onboarding,-66.37,87.34,-0.76,0.45,-237.57,104.83
Having ExpertAssist Effect on MTS Bundle,-6.94,37.02,-0.19,0.85,-79.52,65.63
Constant,"1,031.48",24.95,41.34,0.00,982.57,"1,080.39"
TestGroup,106.39,25.67,4.14,0.00,56.06,156.71
WeeksAgo,-0.28,8.91,-0.03,0.97,-17.76,17.19
VZN_Onboarding_2628,225.05,34.49,6.53,0.00,157.45,292.64


# Regression of LogHandleTime with prediction

In [6]:
df = dfp.copy()
df = add_conditionals(df)


# Look at only 2 queue types
df['Having ExpertAssist Effect on Onboarding']=df['VZN_Onboarding_2628']*df['ExpertAssistOnForThisUser']
df['Having ExpertAssist Effect on MTS Bundle']=df['VZN_MTS_Bundle_EN_2233']*df['ExpertAssistOnForThisUser']
df['Onboarding Post Launch']=df['VZN_Onboarding_2628']*df['AfterExpertAssistLaunch']
# df['MTS Bundle Post Launch']=df['VZN_MTS_Bundle_EN_2233']*df['AfterExpertAssistLaunch']
df['Week']=df['Week'].astype(int)
df['WeeksAgo']=-(df['Week']-df['Week'].max())
# display(df[['Date','Employee','TaskQueue',
#             'TestGroup','AfterExpertAssistLaunch','ExpertAssistOnForThisUser',
#             'HandleTime','Having ExpertAssist Effect on Onboarding',
#             'Having ExpertAssist Effect on MTS Bundle','WeeksAgo']].tail(25),df.columns)
model = sm.OLS(
    endog=df['Log(HandleTime)'],
    # endog=df['HandleTime'],
    exog=df[[
        'Having ExpertAssist Effect on Onboarding',
        'Having ExpertAssist Effect on MTS Bundle','Constant','TestGroup','WeeksAgo',
        "VZN_Onboarding_2628"
]]).fit()
display(np.exp(model.predict(exog=[1,0,1,1,1,1]))-np.exp(model.predict(exog=[0,0,1,1,1,1])))
display(np.exp(model.predict(exog=[0,1,1,1,1,0]))-np.exp(model.predict(exog=[0,0,1,1,1,0])))
display(model.summary())
del df, model

array([-115.97534653])

array([-36.03194685])

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        Log(HandleTime)   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     16.20
Date:                Tue, 10 Oct 2023   Prob (F-statistic):           5.92e-16
Time:                        13:06:25   Log-Likelihood:                -15238.
No. Observations:               10990   AIC:                         3.049e+04
Df Residuals:                   10984   BIC:                         3.053e+04
Df Model:                           5                                         
Covariance Type:            nonrobust                                         
============================================================================================================
                                               coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------------
Having ExpertAssist Effect on Onboarding    -0.1202      0.089     -1.346      0.178      -0.295       0.055
Having ExpertAssist Effect on MTS Bundle    -0.0436      0.038     -1.151      0.250      -0.118       0.031
Constant                                     6.6036      0.026    258.857      0.000       6.554       6.654
TestGroup                                    0.1551      0.026      5.909      0.000       0.104       0.207
WeeksAgo                                    -0.0192      0.009     -2.111      0.035      -0.037      -0.001
VZN_Onboarding_2628                          0.1919      0.035      5.442      0.000       0.123       0.261
==============================================================================
Omnibus:                     2374.678   Durbin-Watson:                   2.019
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             7071.489
Skew:                          -1.121   Prob(JB):                         0.00
Kurtosis:                       6.228   Cond. No.                         28.8
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [7]:
df = dfp.copy()
df = add_conditionals(df)
px.box(df[df['AfterExpertAssistLaunch']==1],x='TestGroup',y='HandleTime').show()
display(df[['Log(HandleTime)','HandleTime']].describe(),df[df['HandleTime']==0])
del df

,Log(HandleTime),HandleTime
count,"10,990.00","10,990.00"
mean,6.62,"1,085.34"
std,0.97,950.13
mad (mean),0.72,669.97
mad (median),0.57,547.34
min,0.00,0.00
25%,6.13,461.00
50%,6.72,832.00
75%,7.24,"1,397.00"
max,9.33,"11,253.00"


,Date,Employee,callNo,TaskQueue,HandleTime,TestGroup,AfterExpertAssistLaunch,ExpertAssistOnForThisUser,Day,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday,Week,Week 38,Week 37,Week 39,Week 40,Week 41,VZN_MTS_Bundle_EN_2233,VZN_Onboarding_2628,HandleTimeZ,Log(HandleTime),Log(HandleTime)Z,Constant
3539,2023-09-25,568568,WT66f5994f01f0847c98926571c61e50ee,VZN_MTS_Bundle_EN_2233,0.00,0,1,0,0,1,0,0,0,0,0,0,39,0,0,1,0,0,1,0,-1.14,0.00,-6.81,1
3952,2023-09-12,581694,WTe9ca7b69335a642ec2f52d014711822e,VZN_MTS_Bundle_EN_2233,0.00,0,0,0,1,0,1,0,0,0,0,0,37,0,1,0,0,0,1,0,-1.14,0.00,-6.81,1
4987,2023-09-25,547381,WT32971ca05eaa82e18cd1e8e591ce8508,VZN_MTS_Bundle_EN_2233,0.00,0,1,0,0,1,0,0,0,0,0,0,39,0,0,1,0,0,1,0,-1.14,0.00,-6.81,1
7113,2023-09-18,576565,WTe6bf7d93da615d1bbaf0b10158af444b,VZN_MTS_Bundle_EN_2233,0.00,0,0,0,0,1,0,0,0,0,0,0,38,1,0,0,0,0,1,0,-1.14,0.00,-6.81,1
10619,2023-09-12,575731,WTcabe350bea26e2537ac1e7e927b955bd,VZN_MTS_Bundle_EN_2233,0.00,0,0,0,1,0,1,0,0,0,0,0,37,0,1,0,0,0,1,0,-1.14,0.00,-6.81,1
